# Calculation of the need of the bike-ways network

## Plotting the basic geographical information

In [1]:
sos_north = 19.6
sos_west = -99.367
sos_south = 19.2
sos_east = -98.933

In [2]:
import geoplotlib
from geoplotlib.utils import BoundingBox
from geoplotlib.utils import read_csv

BoundingBox.CDMX = BoundingBox(sos_north,sos_west,sos_south,sos_east)

In [3]:
data_dots = read_csv('cdmx_distritos/cdmx_centroides_trim.csv')
geoplotlib.shapefiles('cdmx_distritos/O-D_2017',color=[0,0,255],linewidth=1)
geoplotlib.dot(data_dots, point_size = 3)
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(BoundingBox.CDMX)
geoplotlib.show()

## Computing the weights of the complete graph

***
### Considering the number of users

I will supose for this part that every user of any transportation mode is a potential future user of the bike-ways network that is our object of design. For that I will import the data from the origin-destination study to make a weighted graph, with an initial weight term as a function of the ammount of potential users that every link might have. Let's extract o-d data from the csv file into a DataFrame

In [4]:
using CSV, DataFrames

od_df = CSV.File("EOD17.csv") |> DataFrame

,1 Centro Histórico
,Int64
1,70895
2,13314
3,15254
4,10625
5,10365
6,9106
7,10101
8,5593
9,2552


Now for processing this data I will put it into a matrix (the adjacency matrix) supposing that **the weight of one path with a potential number of users is inversely proportional to the number of users**, i. e. the more potential users a link will have the less costful will be to build it.

$$w_{1} \propto \frac{1}{\text{no. of users}}$$

In [5]:
adj_matrix = Array{Float64}(undef, 85, 85)

for ci in 1:85
    c = coalesce.(od_df[:][ci], 0.0)
    for ri in 1:85
        if ci == ri
            adj_matrix[ri,ci] = 0
        else 
            adj_matrix[ri,ci] = 1/c[ri]
        end
    end 
end 

adj_matrix

┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[7]:4
└ @ Core ./In[7]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[7]:4
└ @ Core ./In[7]:4


85×85 Array{Float64,2}:
 0.0          8.01667e-5     5.93296e-5   …    0.000315557  0.000376648
 7.51089e-5   0.0            5.04236e-5        0.000880282  0.00136426 
 6.55566e-5   5.03018e-5     0.0             Inf            0.00357143 
 9.41176e-5   0.000200763    0.0001485         0.00266667   0.00131062 
 9.64785e-5   0.000154036    0.000561482       0.0035461    0.00137174 
 0.000109818  0.000235183    0.00132275   …    0.000449236  0.00071582 
 9.90001e-5   0.000190114    0.000254323       0.000844595  0.000875657
 0.000178795  7.30994e-5     0.000282885       0.00120337   0.000571755
 0.00039185   0.000192864    0.00162602        0.0015949    0.00152439 
 0.000171468  0.000272183    0.0015083         0.00161031   0.000613874
 0.000362713  0.000769231    0.0037037    …    0.00346021   0.00218818 
 0.000147319  0.000405186    0.00280112        0.000600601  0.000738552
 0.00020008   0.000216591    0.00169779        0.000892061  0.000974659
 ⋮                                      

I will take the constant of proportionality so that the biggest (finite) weight possible is 1, this can be modified according to the importance of this variable into the total weights.

In [6]:
M = adj_matrix[1,1]
for ci in 1:85
    for ri in 1:84
        if M < adj_matrix[ri+1,ci] && adj_matrix[ri+1,ci] != Inf
            M = adj_matrix[ri+1,ci]
        end
    end 
end 

weight₁=adj_matrix/M

85×85 Array{Float64,2}:
 0.0          0.000881834    0.000652625  …    0.00347113  0.00414313
 0.000826198  0.0            0.000554659       0.0096831   0.0150068 
 0.000721122  0.00055332     0.0             Inf           0.0392857 
 0.00103529   0.00220839     0.0016335         0.0293333   0.0144168 
 0.00106126   0.00169439     0.00617631        0.0390071   0.0150892 
 0.00120799   0.00258702     0.0145503    …    0.0049416   0.00787402
 0.001089     0.00209125     0.00279756        0.00929054  0.00963222
 0.00196674   0.000804094    0.00311174        0.0132371   0.00628931
 0.00431034   0.0021215      0.0178862         0.0175439   0.0167683 
 0.00188615   0.00299401     0.0165913         0.0177134   0.00675261
 0.00398984   0.00846154     0.0407407    …    0.0380623   0.02407   
 0.00162051   0.00445705     0.0308123         0.00660661  0.00812408
 0.00220088   0.0023825      0.0186757         0.00981267  0.0107212 
 ⋮                                        ⋱                       

***
### Considering the distance between nodes

Now I will import the data obtained with qGIS from the locations of the centroids of *Distritos de Tránsito* to calculate the second term of the weight series, as a function of the distance between each pair of nodes. I am supposing that a node is well connected when there is a path going trough (or pretty near) its centroid.

In [7]:
centroids_df = CSV.File("cdmx_distritos/cdmx_centroides_trim.csv") |> DataFrame

,Nombre,lat,lon
,String,Float64,Float64
1,Centro Histórico,19.4334,-99.1374
2,Buenavista-Reforma,19.4428,-99.1583
3,Tlatelolco,19.4526,-99.1352
4,Morelos,19.4363,-99.1163
5,"Moctezuma, Terminal de Autobuses de Oriente (TAPO)",19.4375,-99.1006
6,Balbuena,19.4129,-99.1195
7,Obrera,19.4145,-99.1448
8,Condesa,19.4114,-99.1683
9,Nápoles,19.3934,-99.1724


Calculating the distance between each pair of nodes and storing it in a matrix:

In [8]:
centroid_array = Array{Float64}(undef, 85, 2)

for ci in 2:3
    c = coalesce.(centroids_df[:][ci], 0.0)
    for ri in 1:85 
        centroid_array[ri,ci-1] = c[ri]
    end 
end

centroid_list = []

for ri in 1:85
    push!(centroid_list, [centroid_array[ri,1],centroid_array[ri,2]]) 
end

centroid_list

┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[25]:4
└ @ Core ./In[25]:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[25]:4
└ @ Core ./In[25]:4


85-element Array{Any,1}:
 [19.4334, -99.1374]
 [19.4428, -99.1583]
 [19.4526, -99.1352]
 [19.4363, -99.1163]
 [19.4375, -99.1006]
 [19.4129, -99.1195]
 [19.4145, -99.1448]
 [19.4114, -99.1683]
 [19.3934, -99.1724]
 [19.394, -99.1469] 
 [19.3916, -99.1304]
 [19.3947, -99.1107]
 [19.3722, -99.1257]
 ⋮                  
 [19.2842, -99.0372]
 [19.2502, -98.9896]
 [19.3014, -99.0283]
 [19.319, -99.0597] 
 [19.3331, -99.047] 
 [19.3312, -99.0279]
 [19.3373, -99.0058]
 [19.3169, -98.9917]
 [19.3782, -99.0184]
 [19.3685, -99.0224]
 [19.3537, -99.0393]
 [19.3798, -99.0501]

In [9]:
using LinearAlgebra

distance_matrix = Array{Float64}(undef, 85, 85)

for ri in 1:85
    for ci in 1:85
        distance_matrix[ri,ci] = norm(centroid_list[ci] - centroid_list[ri])
    end
end

distance_matrix

85×85 Array{Float64,2}:
 0.0        0.0229374  0.0193748  …  0.132096   0.126393   0.102377 
 0.0229374  0.0        0.0250832     0.154953   0.148681   0.125163 
 0.0193748  0.0250832  0.0           0.140818   0.137825   0.111991 
 0.0212961  0.0424989  0.0249799     0.115912   0.112968   0.0870112
 0.0370396  0.0579678  0.037813      0.104348   0.103831   0.0766187
 0.0272301  0.049025   0.0427651  …  0.106792   0.0996627  0.0768058
 0.0202749  0.0313507  0.0393093     0.130814   0.121784   0.100812 
 0.0379106  0.0329021  0.0528242     0.152161   0.141374   0.122341 
 0.0531572  0.0513881  0.0699468     0.152093   0.138907   0.123012 
 0.0404436  0.0500613  0.0597165     0.127106   0.114904   0.0977673
 0.0423468  0.0582857  0.0612136  …  0.110521   0.0987193  0.0811537
 0.0470071  0.0676821  0.0629291     0.0921573  0.0823486  0.0623641
 0.0622819  0.0777445  0.0809959     0.103456   0.0884298  0.0760036
 ⋮                                ⋱                                 
 0.179719 

The last matrix represents **the cost of each edge due to its longitude**, making the assuption that the cost of a path is indeed proportional to the distance the path covers.

$$w_{2} \propto {\text{distance}}$$

Once again, I will take the constant of proportionality so that the biggest (finite) weight possible is 1, this can be modified according to the importance of this variable into the total weights.

In [10]:
M = distance_matrix[1,1]
for ci in 1:85
    for ri in 1:84
        if M < distance_matrix[ri+1,ci] && distance_matrix[ri+1,ci] != Inf
            M = distance_matrix[ri+1,ci]
        end
    end 
end 

weight₂ = distance_matrix/M

85×85 Array{Float64,2}:
 0.0        0.0525858  0.0444182  …  0.302841   0.289766   0.234708 
 0.0525858  0.0        0.0575052     0.355242   0.340863   0.286945 
 0.0444182  0.0575052  0.0           0.322836   0.315974   0.256748 
 0.0488231  0.0974321  0.0572685     0.265737   0.258989   0.19948  
 0.0849163  0.132896   0.0866894     0.239226   0.23804    0.175655 
 0.0624272  0.112394   0.0980423  …  0.244829   0.228485   0.176083 
 0.0464818  0.0718741  0.0901198     0.299901   0.2792     0.23112  
 0.0869132  0.0754308  0.121104      0.348842   0.324111   0.280476 
 0.121867   0.117811   0.160359      0.348684   0.318455   0.282015 
 0.0927201  0.11477    0.136905      0.291401   0.263426   0.224139 
 0.0970834  0.133625   0.140337   …  0.253379   0.226322   0.186051 
 0.107767   0.155167   0.14427       0.211278   0.188791   0.142975 
 0.142786   0.178235   0.18569       0.237181   0.202732   0.174244 
 ⋮                                ⋱                                 
 0.41202  

***
### Calculating the total
Adding the first and second weight I get the total weight of each possible vertex (considering only this two terms):

In [11]:
total_weight = weight₁ + weight₂

85×85 Array{Float64,2}:
 0.0        0.0534676    0.0450708  …  0.305769     0.293237   0.238851 
 0.053412   0.0          0.0580598     0.363581     0.350546   0.301952 
 0.0451393  0.0580585    0.0           0.349997   Inf          0.296034 
 0.0498584  0.0996405    0.058902      0.273976     0.288322   0.213897 
 0.0859775  0.13459      0.0928657     0.249864     0.277047   0.190744 
 0.0636352  0.114981     0.112593   …  0.258841     0.233426   0.183957 
 0.0475708  0.0739653    0.0929174     0.306763     0.28849    0.240752 
 0.0888799  0.0762349    0.124215      0.364489     0.337348   0.286765 
 0.126178   0.119933     0.178245      0.380294     0.335998   0.298783 
 0.0946063  0.117764     0.153496      0.327585     0.281139   0.230892 
 0.101073   0.142086     0.181078   …  0.345046     0.264384   0.210121 
 0.109388   0.159624     0.175082      0.223106     0.195397   0.151099 
 0.144987   0.180618     0.204365      0.258133     0.212545   0.184966 
 ⋮                         

## Imposing the geographical constrictions

Now I will impose the geographical constrictions between pairs, allowing to travel between geographically adjacent pairs of *distritos*. The information about neighoring pairs was obtained using qGIS and expanded by hand.

In [12]:
constricted_pairs_df = CSV.File("cdmx_distritos/vecinos_expand.csv") |> DataFrame

connected_pairs = Array{Int64}(undef, length(constricted_pairs_df[:][1]), 2)

for ci in 1:2
    c = coalesce.(constricted_pairs_df[:][ci], 0)
    for ri in 1:length(c) 
        connected_pairs[ri,ci] = c[ri]
    end 
end

olat = []
olon = []
dlat = []
dlon = []

for i in 1:length(constricted_pairs_df[:][1])
    index_o = connected_pairs[i,1]
    index_d = connected_pairs[i,2]
    push!(olat, centroid_array[index_o,1])
    push!(olon, centroid_array[index_o,2])
    push!(dlat, centroid_array[index_d,1])
    push!(dlon, centroid_array[index_d,2])
end

OD_constricted_df = DataFrame(o_lat = olat, o_lon = olon, d_lat = dlat, d_lon = dlon) 

┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[49]:2
└ @ Core In[49]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[49]:2
└ @ Core In[49]:2
┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[49]:6
└ @ Core ./In[49]:6
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[49]:6
└ @ Core ./In[49]:6
┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[49]:16
└ @ Core ./In[49]:16
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[49]:16
└ @ Core ./In[49]:16


,o_lat,o_lon,d_lat,d_lon
,Any,Any,Any,Any
1,19.4334,-99.1374,19.4129,-99.1195
2,19.4334,-99.1374,19.4363,-99.1163
3,19.4334,-99.1374,19.4526,-99.1352
4,19.4334,-99.1374,19.4428,-99.1583
5,19.4334,-99.1374,19.4114,-99.1683
6,19.4334,-99.1374,19.4145,-99.1448
7,19.4428,-99.1583,19.4334,-99.1374
8,19.4428,-99.1583,19.4526,-99.1352
9,19.4428,-99.1583,19.4901,-99.139


In [13]:
CSV.write("coordenadas_graf.csv",OD_constricted_df;delim=",")

"coordenadas_graf.csv"

Then I will plot everything to see that it is in fact correct

In [14]:
data_dots = read_csv('cdmx_distritos/cdmx_centroides_trim.csv')
data_graph = read_csv('coordenadas_graf.csv')
geoplotlib.shapefiles('cdmx_distritos/O-D_2017',color=[0,0,255],linewidth=1)
geoplotlib.set_smoothing(True)
geoplotlib.dot(data_dots, point_size = 3)
geoplotlib.graph(data_graph,'o_lat','o_lon','d_lat','d_lon',linewidth=2,color='Dark2')
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(BoundingBox.CDMX)
geoplotlib.show()

The next step is to modify the weight matrix so that the forbidden connections weight infinite.

In [15]:
total_constricted_weight = fill(Inf,(85,85))

for i in 1:length(constricted_pairs_df[:][1])
    index_o = connected_pairs[i,1]
    index_d = connected_pairs[i,2]
    total_constricted_weight[index_o,index_d] = total_weight[index_o,index_d]
end

for i in 1:85
    total_constricted_weight[i,i] = 0
end

total_constricted_weight

┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[61]:2
└ @ Core ./In[61]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[61]:2
└ @ Core ./In[61]:2


85×85 Array{Float64,2}:
   0.0          0.0534676    0.0450708  …  Inf          Inf        
   0.053412     0.0          0.0580598     Inf          Inf        
   0.0451393    0.0580585    0.0           Inf          Inf        
   0.0498584  Inf            0.058902      Inf          Inf        
 Inf          Inf          Inf             Inf          Inf        
   0.0636352  Inf          Inf          …  Inf          Inf        
   0.0475708    0.0739653  Inf             Inf          Inf        
   0.0888799    0.0762349  Inf             Inf          Inf        
 Inf          Inf          Inf             Inf          Inf        
 Inf          Inf          Inf             Inf          Inf        
 Inf          Inf          Inf          …  Inf          Inf        
 Inf          Inf          Inf             Inf          Inf        
 Inf          Inf          Inf             Inf          Inf        
   ⋮                                    ⋱                          
 Inf          Inf       

And that is the weight matrix of a complete graph representing all the possible connections in the city.

## Example calculation of the shortest path tree

This is an example calcuation using LightGraphs, of the tree with node 71 (milpa alta) as the source node. The shortest path in this case must be understood as the **path that is used by the most people in each vertex and connects the nearest neigboring nodes**. 

In [21]:
using LightGraphs

ds = dijkstra_shortest_paths(complete_digraph(85), 8, total_constricted_weight)

test_d = ds.parents

olat71 = []
olon71 = []
dlat71 = []
dlon71 = []

for i in 1:85
    index_d = test_d[i]
    push!(olat71, centroid_array[i,1])
    push!(olon71, centroid_array[i,2])
    if index_d != 0
        push!(dlat71, centroid_array[index_d,1])
        push!(dlon71, centroid_array[index_d,2])
    else
        push!(dlat71, centroid_array[71,1])
        push!(dlon71, centroid_array[71,2])
    end
end

shortest_path_tree_71 = DataFrame(o_lat = olat71, o_lon = olon71, d_lat = dlat71, d_lon = dlon71)

,o_lat,o_lon,d_lat,d_lon
,Any,Any,Any,Any
1,19.4334,-99.1374,19.4114,-99.1683
2,19.4428,-99.1583,19.4114,-99.1683
3,19.4526,-99.1352,19.4334,-99.1374
4,19.4363,-99.1163,19.4334,-99.1374
5,19.4375,-99.1006,19.4363,-99.1163
6,19.4129,-99.1195,19.4145,-99.1448
7,19.4145,-99.1448,19.4114,-99.1683
8,19.4114,-99.1683,19.1395,-99.0511
9,19.3934,-99.1724,19.4114,-99.1683


In [22]:
CSV.write("spt_71.csv",shortest_path_tree_71;delim=",")

"spt_71.csv"

In [23]:
data_dots = read_csv('cdmx_distritos/cdmx_centroides_trim.csv')
data_graph = read_csv('spt_71.csv')
geoplotlib.shapefiles('cdmx_distritos/O-D_2017',color=[0,0,255],linewidth=1)
geoplotlib.set_smoothing(True)
geoplotlib.dot(data_dots, point_size = 3)
geoplotlib.graph(data_graph,'o_lat','o_lon','d_lat','d_lon',linewidth=2,color='Dark2')
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(BoundingBox.CDMX)
geoplotlib.show()

A second example is developed below using only the weight term associated with the number of users trough a path. The shortest path in this case must be understood as the **path that is used by the most people in each vertex**.

In [24]:
users_constricted_weight = fill(Inf,(85,85))

for i in 1:length(constricted_pairs_df[:][1])
    index_o = connected_pairs[i,1]
    index_d = connected_pairs[i,2]
    users_constricted_weight[index_o,index_d] = weight₁[index_o,index_d]
end

for i in 1:85
    users_constricted_weight[i,i] = 0
end

ds = dijkstra_shortest_paths(complete_digraph(85), 8, users_constricted_weight)

test_d = ds.parents

olat71 = []
olon71 = []
dlat71 = []
dlon71 = []

for i in 1:85
    index_d = test_d[i]
    push!(olat71, centroid_array[i,1])
    push!(olon71, centroid_array[i,2])
    if index_d != 0
        push!(dlat71, centroid_array[index_d,1])
        push!(dlon71, centroid_array[index_d,2])
    else
        push!(dlat71, centroid_array[71,1])
        push!(dlon71, centroid_array[71,2])
    end
end

users_shortest_path_tree_71 = DataFrame(o_lat = olat71, o_lon = olon71, d_lat = dlat71, d_lon = dlon71)
CSV.write("uspt_71.csv",users_shortest_path_tree_71;delim=",")

┌ Warning: `getindex(df::DataFrame, ::Colon)` is deprecated, use `df[:, :]` instead.
│   caller = top-level scope at In[97]:2
└ @ Core ./In[97]:2
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[97]:2
└ @ Core ./In[97]:2


"uspt_71.csv"

In [25]:
%with python3
data_dots = read_csv('cdmx_distritos/cdmx_centroides_trim.csv')
data_graph = read_csv('uspt_71.csv')
geoplotlib.shapefiles('cdmx_distritos/O-D_2017',color=[0,0,255],linewidth=1)
geoplotlib.set_smoothing(True)
geoplotlib.dot(data_dots, point_size = 3)
geoplotlib.graph(data_graph,'o_lat','o_lon','d_lat','d_lon',linewidth=2,color='Dark2')
geoplotlib.set_smoothing(True)
geoplotlib.set_bbox(BoundingBox.CDMX)
geoplotlib.show()

## Comparison of two examples

<table>
<tr>
<td><h3> Shortest path using total weight </h3></td>
<td><h3> Shortest path using number of users </h3></td>
</tr>
<tr>
<td> <img src="spt_71.png" style="width: 450px;"/> </td>
<td> <img src="uspt_71.png" style="width: 450px;"/> </td>
</tr></table>